# 🌇 FashionMNIST 조건부 생성(cGAN) 구현 및 개선 실험


## 1. 프로젝트 목적
본 실습의 목적은 FashionMNIST(10개 클래스) 데이터셋에서 Conditional GAN(cGAN) 을 학습하여,
주어진 클래스 레이블(예: T-shirt, Trouser, Sneaker 등)을 조건으로 해당 클래스의 이미지를 생성하는 모델을 구축하는 것이다.

- 일반 GAN이 “그럴듯한 이미지”를 생성하는 문제라면,
- cGAN은 ‘그럴듯함’과 동시에 ‘조건 일치(레이블 반영)’ 를 만족해야 한다.
따라서 생성 품질뿐 아니라, 레이블-이미지 매칭이 실제로 성립하는지를 핵심 평가 대상으로 둔다.


## 2. 데이터
- 데이터셋: FashionMNIST (torchvision.datasets.FashionMNIST)
- 이미지: 28×28 흑백(1채널), 10개 클래스
- 학습/평가: train 60,000 / test 10,000


## 3. 전체 로드맵

1) 데이터 로드 및 전처리  
- ToTensor 및 Normalize를 적용하여 입력 범위를 모델 출력 범위(tanh)와 정합되도록 설정한다.

2) Baseline cGAN 구현 및 재현  
- 조건 정보를 (noise + label embedding) 형태로 입력에 결합하는 표준적 cGAN 구조를 구현하고,
- 학습 로그(D(x), D(G(z)), loss)의 변화를 관찰하며 학습 안정성과 한계를 확인한다.

3) 개선 실험(구조 및 학습 안정화)  
- Generator/Discriminator 구조를 더 표현력 있는 형태로 변경(Residual block 기반)
- 손실함수 및 학습 전략을 안정화 방향으로 조정(예: LSGAN 스타일 MSE, AdamW, LR scheduler, label smoothing 등)

4) 평가 및 분석  
- 정성 평가: 클래스별 샘플 그리드로 레이블 반영 여부 및 품질 확인
- 정량 평가(선택): FID/IS를 이용하여 체크포인트별 생성 품질 변화를 비교한다.


## 4. 평가 기준
- 생성 품질(선명도/다양성): 정성 평가 및 FID/IS(선택)
- 조건 반영(클래스 일치): 클래스별 생성 결과를 고정 포맷으로 반복 점검하여 확인한다.

✅ 최종 목표는 “돌아가는 GAN”이 아니라,
"조건부 생성이 성립하는지"와 "개선 전략이 실제로 품질을 올리는지"를 근거(로그/샘플/지표)로 설명하는 것이다.


In [ ]:
▫️임포트 & 디바이스 & 시드 고정

In [ ]:
!pip -q install torch torchvision numpy matplotlib scipy

import os
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms
from torchvision.utils import make_grid
import matplotlib.pyplot as plt

# 재현성(reproducibility)
# GAN은 초기값에 따라 결과가 크게 흔들릴 수 있어서
# 실험 비교를 하려면 seed 고정이 거의 필수
def seed_everything(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

    # 아래 옵션은 속도가 조금 느려질 수 있지만, 재현성을 올립니다.
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed_everything(42)
print("Seed fixed to 42")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using Device:", device)


▫️데이터 로드/전처리

- FashionMNIST는 흑백 28x28 이미지입니다.
- GAN에서 자주 쓰는 전처리:
    - ToTensor()로 [0, 1] 범위 텐서로 만들고 Normalize((0.5,), (0.5,))로 [-1, 1]로 바꿉니다.

- 왜 [-1, 1]로 바꾸냐?
    - Generator 출력에 tanh()를 쓰면 출력 범위가 [-1, 1]이라서 입력 데이터 범위도 맞추는 게 학습 안정에 유리하다.

◻️ transform + DataLoader

In [ ]:
# 베이스라인 하이퍼파라미터
image_size = 28
num_classes = 10
latent_dim = 100
batch_size = 64
num_epochs = 200
lr = 0.0002
beta1, beta2 = 0.5, 0.999

train_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))  # [0,1] -> [-1,1]
])

test_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_dataset = torchvision.datasets.FashionMNIST(
    root="./data", train=True, download=True, transform=train_transforms
)
test_dataset = torchvision.datasets.FashionMNIST(
    root="./data", train=False, download=True, transform=test_transforms
)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader  = torch.utils.data.DataLoader(test_dataset,  batch_size=batch_size, shuffle=False)

idx_to_class = {i: name for i, name in enumerate(train_dataset.classes)}
print("FashionMNIST classes:", idx_to_class)


▫️베이스라인 Generator/Discriminator

- 베이스라인은 “조건부 생성”의 가장 단순한 형태를 택합니다.
    - Generator:
        - 입력: noise z + label embedding
        - 출력: 1채널(흑백) 28x28 이미지

- Discriminator:
    - 입력: 이미지 + label 채널(또는 embedding을 채널로 확장)
    - 출력: 진짜/가짜 확률(0~1) → Sigmoid + BCE

- 이 구조의 장점:
    - 구현이 쉽고 디버깅이 쉽다(미션 baseline으로 적합)

- 단점:
    - 구조가 얕으면 “선명도/다양성”이 한계가 있을 수 있음
    - 조건 정보(레이블)가 이미지 생성에 충분히 강하게 박히지 않을 수 있음

In [ ]:
# 베이스라인 모델 정의(BCE버전)
class Generator(nn.Module):
    def __init__(self):
        super().__init__()
        # WHY: 레이블을 벡터로 바꾸는 임베딩. one-hot보다 학습 유연성이 좋습니다.
        self.label_emb = nn.Embedding(num_classes, 10)

        self.model = nn.Sequential(
            nn.Linear(latent_dim + 10, 128),
            nn.ReLU(True),

            nn.Linear(128, 256),
            nn.ReLU(True),

            nn.Linear(256, image_size * image_size),
            nn.Tanh()  # [-1,1] 범위로 맞춤
        )

    def forward(self, z, labels):
        # torch.cat: 마지막 차원에서 (noise + label_vector) 연결
        # WHY: 조건 정보를 "입력 자체"에 섞어주는 가장 단순한 방식
        x = torch.cat([z, self.label_emb(labels)], dim=1)
        out = self.model(x)
        return out.view(-1, 1, image_size, image_size)


class Discriminator(nn.Module):
    def __init__(self):
        super().__init__()
        # 레이블을 1차원으로 임베딩해서 이미지 채널처럼 붙이는 방식
        self.label_emb = nn.Embedding(num_classes, 1)

        self.model = nn.Sequential(
            nn.Conv2d(1 + 1, 64, kernel_size=3, stride=2, padding=1),  # 28 -> 14
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1),    # 14 -> 7
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Flatten(),
            nn.Linear(128 * (image_size // 4) * (image_size // 4), 1),
            nn.Sigmoid()  # BCE를 쓰기 때문에 확률로 변환
        )

    def forward(self, img, labels):
        b = img.size(0)
        label = self.label_emb(labels)         # (B,1)
        label = label.view(b, 1, 1, 1)         # (B,1,1,1)
        label = label.expand(b, 1, image_size, image_size)  # (B,1,28,28)
        x = torch.cat([img, label], dim=1)     # (B,2,28,28)
        return self.model(x)


▫️ 학습 루프(D(x), D(G(z))를 같이 보기)

- GAN은 loss만 보면 감이 잘 안 옵니다. 그래서 보통 아래를 같이 봅니다.
    - D(x): 진짜 이미지를 진짜라고 보는 평균 점수
    - D(G(z)): 가짜를 진짜라고 보는 평균 점수

- 이 값들이
    - D(x)만 1.0에 가깝고 D(G(z))가 0에 박혀 있으면 → 판별자만 압승(생성자 학습이 막힘)
    - 둘 다 0.5 근처에서 오래 머무르면 → 둘 다 “헷갈리기만 하고” 진짜 생성이 좋아졌다고 보기 애매

In [ ]:
# 베이스라인 학습 함수
adversarial_loss = nn.BCELoss()

def train_one_epoch(generator, discriminator,
                    optimizer_G, optimizer_D,
                    train_loader, device):
    generator.train()
    discriminator.train()

    g_loss_sum = 0.0
    d_loss_sum = 0.0
    d_real_sum = 0.0
    d_fake_sum = 0.0
    n = 0

    for imgs, labels in train_loader:
        b = imgs.size(0)
        imgs, labels = imgs.to(device), labels.to(device)

        valid = torch.ones(b, 1, device=device)
        fake  = torch.zeros(b, 1, device=device)


        # Discriminator 학습
        optimizer_D.zero_grad()

        real_pred = discriminator(imgs, labels)
        d_real_loss = adversarial_loss(real_pred, valid)

        z = torch.randn(b, latent_dim, device=device)
        gen_labels = torch.randint(0, num_classes, (b,), device=device)
        gen_imgs = generator(z, gen_labels)

        fake_pred = discriminator(gen_imgs.detach(), gen_labels)
        d_fake_loss = adversarial_loss(fake_pred, fake)

        d_loss = 0.5 * (d_real_loss + d_fake_loss)
        d_loss.backward()
        optimizer_D.step()


        # Generator 학습
        optimizer_G.zero_grad()

        pred = discriminator(gen_imgs, gen_labels)
        g_loss = adversarial_loss(pred, valid)       # "가짜를 진짜로 속이는" 목표
        g_loss.backward()
        optimizer_G.step()

        g_loss_sum += g_loss.item()
        d_loss_sum += d_loss.item()
        d_real_sum += real_pred.mean().item()
        d_fake_sum += fake_pred.mean().item()
        n += 1

    return {
        "g_loss": g_loss_sum / n,
        "d_loss": d_loss_sum / n,
        "d_real": d_real_sum / n,
        "d_fake": d_fake_sum / n
    }


'''
에폭을 많이 돌려도 “조건부로 잘 생성된다”는 확신이 약했다.
학습 로그에서 D(x), D(G(z))가 크게 유의미하게 변하지 않는 구간이 길었다.
즉, 생성자가 판별자를 “확실히 속이며 좋아진다”는 느낌이 약해서
단순히 에폭만 늘리는 접근은 효율이 떨어질 수 있다.
그래서 개선 실험에서는 “구조 자체 + 학습 안정화”를 동시에 바꿔서 해볼 예정
'''

In [ ]:
▫️개선 실험 계획

- 시드 고정(비교 실험 필수)
- 변수는 앞에서 한 번에 관리(실험 반복 편하게)
- 데이터 증강은 “좌우반전만” 적용(과한 증강이 해가 될 수 있다고 판단)
- Generator: label_dim=30 + ResNet 블록 사용
- Discriminator: ResNet 블록 + Dropout 추가
- 손실: BCE → MSE(LSGAN 스타일, Sigmoid 제거)
- Optimizer: AdamW + weight_decay
- Scheduler: CosineAnnealingLR
- Label smoothing 적용(smoothing=0.1)
- 체크포인트 저장 후 FID/IS로 정량 평가(가능한 범위에서)

In [ ]:
# 개선 실험 하이퍼파라미터
image_size = 28
num_classes = 10
batch_size = 64
num_epochs = 200
latent_dim = 100

label_dim = 30        # 레이블 임베딩 차원
lr = 0.0001           # 개선 실험은 lr을 더 낮게 시작
betas = (0.5, 0.999)
weight_decay = 0.01

dropout_p = 0.4
smoothing = 0.1

▫️transform / DataLoader
- 개선 실험으로 증강을 최소화

In [ ]:
train_transforms_exp = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.2),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

test_transforms_exp = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_dataset_exp = torchvision.datasets.FashionMNIST(
    root="./data", train=True, download=True, transform=train_transforms_exp
)
test_dataset_exp = torchvision.datasets.FashionMNIST(
    root="./data", train=False, download=True, transform=test_transforms_exp
)

train_loader_exp = torch.utils.data.DataLoader(train_dataset_exp, batch_size=batch_size, shuffle=True)
test_loader_exp  = torch.utils.data.DataLoader(test_dataset_exp,  batch_size=batch_size, shuffle=False)

idx_to_class_exp = {i: name for i, name in enumerate(train_dataset_exp.classes)}
print(f"FashionMNIST classes: {idx_to_class_exp}")


🟨 ResNet 블록을 쓰는 이유

- GAN에서 모델이 얕으면
    - 디테일을 만드는 표현력이 부족하거나 학습이 흔들릴 때 회복이 어려울 수 있습니다.

- ResNet(스킵 연결)의 장점:
    - 깊게 쌓아도 기울기(gradient)가 덜 죽어서 학습이 비교적 안정적
    - Generator에서 업샘플링을 단계적으로 하며 디테일을 만들기 좋음

In [ ]:
# ResNet 기반 Generator/Discriminator

class GeneratorResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.main = nn.Sequential(
            nn.BatchNorm2d(in_channels),
            nn.ReLU(inplace=True),

            # WHY: Upsample로 공간 크기를 키운 뒤 Conv로 디테일을 만든다
            nn.Upsample(scale_factor=2, mode="bilinear", align_corners=False),
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),

            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
        )

        # skip connection도 크기/채널을 맞춰줘야 더하기가 가능
        self.shortcut = nn.Sequential(
            nn.Upsample(scale_factor=2, mode="bilinear", align_corners=False),
            nn.Conv2d(in_channels, out_channels, kernel_size=1)
        )

    def forward(self, x):
        return self.main(x) + self.shortcut(x)


class DiscriminatorResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=2):
        super().__init__()
        self.main = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1),
            nn.LeakyReLU(0.2, inplace=True),
        )

        self.shortcut = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride)
        )

    def forward(self, x):
        return self.main(x) + self.shortcut(x)


class Generator_exp(nn.Module):
    def __init__(self, latent_dim, num_classes, label_dim, image_size):
        super().__init__()
        self.init_size = image_size // 4  # 28 -> 7

        self.label_emb = nn.Embedding(num_classes, label_dim)

        self.l1 = nn.Sequential(
            # WHY: noise와 label embedding을 연결해서 "조건 정보"를 초기에 강하게 주입
            nn.Linear(latent_dim + label_dim, 128 * self.init_size * self.init_size),
            nn.ReLU(inplace=True)
        )

        self.conv_blocks = nn.Sequential(
            GeneratorResidualBlock(128, 128),  # 7 -> 14
            GeneratorResidualBlock(128, 64),   # 14 -> 28
        )

        self.final_layer = nn.Sequential(
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 1, kernel_size=3, padding=1),
            nn.Tanh()  # 데이터가 [-1,1]이므로 출력도 [-1,1]
        )

    def forward(self, noise, labels):
        # dim=-1: 마지막 차원에 붙이겠다는 뜻 (벡터 연결 표준 패턴)
        gen_input = torch.cat((noise, self.label_emb(labels)), dim=-1)

        out = self.l1(gen_input)  # (B, 128*7*7)
        # view: 텐서 모양만 바꾸는 reshape (데이터는 그대로, 형태만 변경)
        out = out.view(out.size(0), 128, self.init_size, self.init_size)

        out = self.conv_blocks(out)
        img = self.final_layer(out)
        return img


class Discriminator_exp(nn.Module):
    def __init__(self, num_classes, dropout_p, image_size):
        super().__init__()
        self.image_features = nn.Sequential(
            DiscriminatorResidualBlock(1, 64, stride=2),   # 28 -> 14
            DiscriminatorResidualBlock(64, 128, stride=2), # 14 -> 7
            nn.Flatten()
        )

        # 레이블 임베딩을 판별자에도 입력으로 준다(조건부 판별)
        self.label_embedding_dim_d = 50
        self.label_embedding = nn.Embedding(num_classes, self.label_embedding_dim_d)

        self.final_layer = nn.Sequential(
            nn.Dropout(p=dropout_p),
            # WHY: 이미지 특징 벡터 + 레이블 벡터를 concat해서 "이 이미지가 이 레이블이 맞는지" 판단하게 함
            nn.Linear(128 * (image_size // 4) * (image_size // 4) + self.label_embedding_dim_d, 1)
            # MSELoss(LSGAN)을 쓸 거라 Sigmoid는 넣지 않음
        )

    def forward(self, img, labels):
        img_feats = self.image_features(img)
        label_feats = self.label_embedding(labels)
        combined = torch.cat([img_feats, label_feats], dim=1)
        validity = self.final_layer(combined)
        return validity


▫️가중치 초기화 + 손실 / 옵티마이저 / 스케줄러

In [ ]:
# WHY: GAN은 초기 가중치에 따라 학습이 크게 흔들릴 수 있어서
# DCGAN 계열에서 자주 쓰는 초기화(Conv: N(0,0.02), BN: N(1,0.02))를 적용
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find("Conv") != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find("BatchNorm2d") != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

generator_exp = Generator_exp(
    latent_dim=latent_dim,
    num_classes=num_classes,
    label_dim=label_dim,
    image_size=image_size
).to(device)

discriminator_exp = Discriminator_exp(
    num_classes=num_classes,
    dropout_p=dropout_p,
    image_size=image_size
).to(device)

generator_exp.apply(weights_init)
discriminator_exp.apply(weights_init)

# 손실: MSELoss (LSGAN 스타일)
# WHY: BCE+Sigmoid는 D가 너무 확신하면 gradient가 약해질 수 있는데,
# MSE 기반은 상대적으로 안정적인 경우가 많아서 실험으로 채택
adversarial_loss_exp = nn.MSELoss().to(device)

optimizer_G_exp = optim.AdamW(
    generator_exp.parameters(),
    lr=lr,
    betas=betas,
    weight_decay=weight_decay
)

optimizer_D_exp = optim.AdamW(
    discriminator_exp.parameters(),
    lr=lr,
    betas=betas,
    weight_decay=weight_decay
)

scheduler_G_exp = optim.lr_scheduler.CosineAnnealingLR(
    optimizer_G_exp, T_max=num_epochs, eta_min=1e-6
)
scheduler_D_exp = optim.lr_scheduler.CosineAnnealingLR(
    optimizer_D_exp, T_max=num_epochs, eta_min=1e-6
)


▫️학습 함수(스무딩 + MSE 버전) + 체크포인트 저장

In [ ]:
def train_one_epoch_exp(generator, discriminator,
                        optimizer_G, optimizer_D,
                        adversarial_loss,
                        train_loader,
                        device, latent_dim, num_classes, smoothing):
    generator.train()
    discriminator.train()

    g_loss_sum = d_loss_sum = d_real_sum = d_fake_sum = 0.0
    n = 0

    for imgs, labels in train_loader:
        b = imgs.size(0)
        imgs, labels = imgs.to(device), labels.to(device)

        # Label smoothing:
        # WHY: D가 "진짜=1.0"에 과확신하는 걸 살짝 막아서 학습을 부드럽게 만들려는 목적
        valid = torch.full((b, 1), 1.0 - smoothing, device=device, dtype=torch.float)
        fake  = torch.full((b, 1), 0.0, device=device, dtype=torch.float)

        z = torch.randn(b, latent_dim, device=device)
        gen_labels = torch.randint(0, num_classes, (b,), device=device)
        gen_imgs = generator(z, gen_labels)

        # 1) Discriminator
        optimizer_D.zero_grad()

        real_validity = discriminator(imgs, labels)
        d_real_loss = adversarial_loss(real_validity, valid)

        fake_validity = discriminator(gen_imgs.detach(), gen_labels)
        d_fake_loss = adversarial_loss(fake_validity, fake)

        d_loss = 0.5 * (d_real_loss + d_fake_loss)
        d_loss.backward()
        optimizer_D.step()

        # 2) Generator
        optimizer_G.zero_grad()

        validity = discriminator(gen_imgs, gen_labels)
        # WHY: G는 D가 gen_imgs를 '진짜(0.9)'라고 보게 만들고 싶다
        g_loss = adversarial_loss(validity, valid)

        g_loss.backward()
        optimizer_G.step()

        g_loss_sum += g_loss.item()
        d_loss_sum += d_loss.item()
        d_real_sum += real_validity.mean().item()
        d_fake_sum += fake_validity.mean().item()
        n += 1

    return {
        "g_loss": g_loss_sum / n,
        "d_loss": d_loss_sum / n,
        "d_real": d_real_sum / n,
        "d_fake": d_fake_sum / n
    }


@torch.no_grad()
def sample_by_class(generator, device, latent_dim, num_classes, idx_to_class, n_per_class=8):
    generator.eval()

    # 클래스별로 n_per_class개씩 생성해서 그리드로 확인
    z = torch.randn(num_classes * n_per_class, latent_dim, device=device)
    labels = torch.tensor(
        [c for c in range(num_classes) for _ in range(n_per_class)],
        device=device
    )

    imgs = generator(z, labels)
    imgs = (imgs + 1) / 2.0  # [-1,1] -> [0,1]로 보기 좋게

    grid = make_grid(imgs, nrow=n_per_class)
    plt.figure(figsize=(12, 6))
    plt.imshow(grid.permute(1, 2, 0).cpu())
    plt.axis("off")
    plt.title("Generated samples by class")
    plt.show()


def train_cgan_exp(num_epochs,
                   generator, discriminator,
                   optimizer_G, optimizer_D,
                   scheduler_G, scheduler_D,
                   adversarial_loss,
                   train_loader,
                   device, latent_dim, num_classes, idx_to_class,
                   smoothing,
                   save_dir="./checkpoints_exp",
                   save_every=10):
    os.makedirs(save_dir, exist_ok=True)

    for epoch in range(1, num_epochs + 1):
        stats = train_one_epoch_exp(
            generator, discriminator,
            optimizer_G, optimizer_D,
            adversarial_loss,
            train_loader,
            device, latent_dim, num_classes,
            smoothing
        )

        # 학습 로그 (노트북 원본 출력 스타일 유지)
        current_lr = optimizer_G.param_groups[0]["lr"]
        print(
            f"Epoch [{epoch}/{num_epochs}] | "
            f"G_loss: {stats['g_loss']:.4f} | D_loss: {stats['d_loss']:.4f} | "
            f"D(x): {stats['d_real']:.4f} | D(G(z)): {stats['d_fake']:.4f} | "
            f"LR: {current_lr:.6f}"
        )

        # 중간 시각화
        if epoch % 10 == 0:
            sample_by_class(generator, device, latent_dim, num_classes, idx_to_class, n_per_class=8)

        # 체크포인트 저장
        if epoch % save_every == 0:
            torch.save(generator.state_dict(), os.path.join(save_dir, f"generator_exp_epoch_{epoch}.pth"))
            torch.save(discriminator.state_dict(), os.path.join(save_dir, f"discriminator_exp_epoch_{epoch}.pth"))
            print(f"Saved models for epoch {epoch}")

        # 스케줄러는 에폭 끝에서 step
        scheduler_G.step()
        scheduler_D.step()


▫️ 개선 실험 학습 실행

In [ ]:
train_cgan_exp(
    num_epochs=num_epochs,
    generator=generator_exp,
    discriminator=discriminator_exp,
    optimizer_G=optimizer_G_exp,
    optimizer_D=optimizer_D_exp,
    scheduler_G=scheduler_G_exp,
    scheduler_D=scheduler_D_exp,
    adversarial_loss=adversarial_loss_exp,
    train_loader=train_loader_exp,
    device=device,
    latent_dim=latent_dim,
    num_classes=num_classes,
    idx_to_class=idx_to_class_exp,
    smoothing=smoothing
)

'''
FID(Fréchet Inception Distance)
- “실제 이미지 분포”와 “생성 이미지 분포”의 거리
- 낮을수록 좋다(가까울수록 진짜와 비슷)

IS(Inception Score)
- 생성 이미지가 선명한가(품질)
- 생성 이미지가 다양하게 나오나(다양성)
- 높을수록 좋다
'''

▫️FID/IS 계산 + 체크포인트 자동 평가

In [ ]:
import torch.nn.functional as F
from torchvision.models import inception_v3
from scipy.linalg import sqrtm
import glob

# Inception 모델 로드(처음 1회만)
inception_model_fid = inception_v3(pretrained=True, transform_input=False).to(device)
inception_model_fid.fc = nn.Identity()  # feature extractor로 사용
inception_model_fid.eval()

inception_model_is = inception_v3(pretrained=True, transform_input=False).to(device)
inception_model_is.eval()

def get_inception_features(images, model, batch_size=32, device=device):
    model.eval()
    n_images = images.shape[0]
    features = []

    # Inception 입력 정규화(일반적인 ImageNet mean/std)
    norm_mean = torch.tensor([0.485, 0.456, 0.406], device=device).view(1,3,1,1)
    norm_std  = torch.tensor([0.229, 0.224, 0.225], device=device).view(1,3,1,1)

    with torch.no_grad():
        for i in range(0, n_images, batch_size):
            batch = images[i:i+batch_size]

            # FashionMNIST는 1채널이므로 3채널로 복제
            if batch.shape[1] == 1:
                batch = batch.repeat(1,3,1,1)

            # Inception 입력 크기 299x299로 업샘플링
            batch = F.interpolate(batch, size=(299,299), mode="bilinear", align_corners=False)
            batch = (batch - norm_mean) / norm_std

            pred = model(batch)
            features.append(pred.cpu().numpy())

    return np.concatenate(features, axis=0)


def calculate_fid(real_images, generated_images, model, batch_size=32, device=device):
    feat_real = get_inception_features(real_images, model, batch_size, device)
    feat_gen  = get_inception_features(generated_images, model, batch_size, device)

    mu_real, sigma_real = feat_real.mean(axis=0), np.cov(feat_real, rowvar=False)
    mu_gen,  sigma_gen  = feat_gen.mean(axis=0),  np.cov(feat_gen,  rowvar=False)

    covmean, _ = sqrtm(sigma_real.dot(sigma_gen), disp=False)
    if np.iscomplexobj(covmean):
        covmean = covmean.real

    fid = np.sum((mu_real - mu_gen) ** 2) + np.trace(sigma_real + sigma_gen - 2 * covmean)
    return float(fid)


def calculate_inception_score(images, model, batch_size=32, splits=10, device=device):
    model.eval()
    n_images = images.shape[0]

    # Inception은 1000-class softmax 출력이므로 확률 분포를 얻는다
    preds = []

    norm_mean = torch.tensor([0.485, 0.456, 0.406], device=device).view(1,3,1,1)
    norm_std  = torch.tensor([0.229, 0.224, 0.225], device=device).view(1,3,1,1)

    with torch.no_grad():
        for i in range(0, n_images, batch_size):
            batch = images[i:i+batch_size]
            if batch.shape[1] == 1:
                batch = batch.repeat(1,3,1,1)
            batch = F.interpolate(batch, size=(299,299), mode="bilinear", align_corners=False)
            batch = (batch - norm_mean) / norm_std

            logits = model(batch)
            prob = torch.softmax(logits, dim=1)
            preds.append(prob.cpu().numpy())

    preds = np.concatenate(preds, axis=0)

    # IS 계산(분할 평균)
    scores = []
    split_size = n_images // splits
    for k in range(splits):
        part = preds[k * split_size : (k + 1) * split_size]
        py = np.mean(part, axis=0)
        kl = part * (np.log(part + 1e-10) - np.log(py + 1e-10))
        kl = np.sum(kl, axis=1)
        scores.append(np.exp(np.mean(kl)))

    return float(np.mean(scores)), float(np.std(scores))


# 실제 이미지 5000장 준비(노트북 원본 출력 흐름)
real_imgs = []
for imgs, _ in test_loader_exp:
    real_imgs.append(imgs)
    if sum(x.size(0) for x in real_imgs) >= 5000:
        break
real_imgs = torch.cat(real_imgs, dim=0)[:5000].to(device)
real_imgs = (real_imgs + 1) / 2.0  # [0,1]

print("--- 모든 체크포인트 자동 평가 시작 ---")
print("  - 5000개의 실제 이미지를 평가용으로 준비했습니다.")

# 체크포인트 목록
ckpts = sorted(glob.glob("./checkpoints_exp/generator_exp_epoch_*.pth"))
print(f"--- 총 {len(ckpts)}개의 체크포인트를 평가합니다. ---\n")

results = {}
for path in ckpts:
    epoch = int(path.split("_")[-1].split(".")[0])
    print(f"Evaluating checkpoint: {os.path.basename(path)} (Epoch {epoch})")

    # 모델 로드
    gen = Generator_exp(latent_dim, num_classes, label_dim, image_size).to(device)
    gen.load_state_dict(torch.load(path, map_location=device))
    gen.eval()

    # 생성 이미지 5000장 만들기
    n_gen = 5000
    z = torch.randn(n_gen, latent_dim, device=device)
    labels = torch.randint(0, num_classes, (n_gen,), device=device)
    gen_imgs = gen(z, labels)
    gen_imgs = (gen_imgs + 1) / 2.0  # [0,1]

    fid = calculate_fid(real_imgs, gen_imgs, inception_model_fid, batch_size=32, device=device)
    is_score, is_std = calculate_inception_score(gen_imgs, inception_model_is, batch_size=32, splits=10, device=device)

    print(f"  -> 결과: FID = {fid:.4f}, IS = {is_score:.4f} ± {is_std:.4f}\n")
    results[epoch] = {"fid": fid, "is": is_score, "is_std": is_std, "path": path}


# 🟨 최종 결론 및 분석

## 1. 실험 요약
본 실험은 FashionMNIST에서 클래스 레이블을 조건으로 이미지를 생성하는 cGAN을 구현하고,
Baseline 재현 이후 구조 및 학습 안정화 기법을 적용하여 생성 품질 변화를 비교·분석하였다.

- Baseline: 단순 CNN 기반 Generator/Discriminator, BCE 기반 손실, 조건 결합(concat) 방식
- 개선 실험: Residual block 기반 구조, LSGAN 스타일(MSE), AdamW + weight decay, CosineAnnealingLR,
  label smoothing 및 dropout 등 안정화 기법을 포함

---

## 2. 정량 결과(기록 기준)
체크포인트 자동 평가 결과(기록된 일부)에서,
학습이 진행될수록 FID가 전반적으로 감소하는 경향을 확인하였다.
- Epoch 10: FID 39.4373, IS 4.0799 ± 0.1412
- Epoch 100: FID 16.8721, IS 4.5606 ± 0.1323
- Epoch 110: FID 18.6463, IS 4.3996 ± 0.1291

해석:
- 최소한 “생성 이미지 분포가 실제 이미지 분포에 가까워지는 방향(FID 감소)”은 관찰되었다.
- 단, FashionMNIST는 자연이미지 도메인과 달라 IS/FID의 절대값보다는 “실험 간 상대 비교”에 의미가 크다.

---

## 3. 핵심 인사이트
(1) 품질 지표 개선은 ‘학습 안정화 패키지’가 만든 결과일 가능성이 크다  
- MSE(LSGAN) + AdamW + 코사인 스케줄 + smoothing/dropout 조합은
  판별자가 과확신하거나 학습이 급격히 불안정해지는 현상을 완화하는 방향으로 작동한다.
- 결과적으로 체크포인트가 진행될수록 FID가 낮아지는 흐름을 만들었다.

(2) cGAN의 본질은 “조건 일치”이며, 여기서 추가 개선이 필요하다  
- 정량 지표가 좋아져도, 레이블과 생성 이미지가 일치하지 않는 사례가 존재한다면
  조건부 생성의 목표를 충분히 달성했다고 보기 어렵다.
- 따라서 다음 개선의 중심은 “조건 정보 주입을 더 강하게/정확하게 만드는 방법”이어야 한다.

---

## 4. 한계 및 다음 실험 방향
- 조건 주입 방식 고도화:
  - 단순 concat을 넘어 projection discriminator, Conditional BatchNorm, FiLM 등으로 확장 검토
- 안정화 기법 재조정:
  - label smoothing 강도, dropout 비율, lr 범위를 스윕하여 최적 조합 탐색
- 평가 프로토콜 고정:
  - 매 N epoch마다 “클래스별 고정 그리드”를 생성해 레이블 일치 여부를 정성 평가 루틴으로 고정

---

## 5. 결론
FashionMNIST 조건부 생성 문제에서 cGAN을 구현하고, 구조/학습 안정화 기법을 통해 생성 품질 개선 흐름(FID 감소)을 확인하였다. 향후에는 정량 지표뿐 아니라 “레이블 조건이 일관되게 반영되는지”를 최우선 목표로 두고, 조건 주입 방식의 구조적 개선을 중심으로 추가 실험을 진행하는 것이 타당할것으로 보임
